In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time


In [13]:
res = requests.get(url)
print("HTTP Status Code:", res.status_code)
html_content = res.text
with open("seattleevents.html", "w") as file:
    file.write(html_content)


HTTP Status Code: 200


In [14]:
soup = BeautifulSoup(html_content, "html.parser")
selector = "div.search-result-preview > div > h3 > a"
a_eles = soup.select(selector)
event_links = [x['href'] for x in a_eles]


In [15]:
from bs4 import BeautifulSoup

event_links_set = set()

for page in range(1, 42):
    time.sleep(1)  # Pause to avoid overloading the server
    url = f"https://visitseattle.org/events/page/{page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        if '/events/' in a['href']:
            event_links_set.add(a['href'])

event_links = list(event_links_set)
print(f"Total unique links collected: {len(event_links)}")


Total unique links collected: 453


In [16]:
from bs4 import BeautifulSoup

event_links_set = set()

for page in range(1, 42):
    time.sleep(1)  # Pause to avoid overloading the server
    url = f"https://visitseattle.org/events/page/{page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        if '/events/' in a['href']:
            event_links_set.add(a['href'])

event_links = list(event_links_set)
print(f"Total unique links collected: {len(event_links)}")


Total unique links collected: 453


In [17]:
# Function to scrape event details and get lat/lon for the location
def scrape_event_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    name = soup.find('h1', class_='page-title').get_text(strip=True) if soup.find('h1', class_='page-title') else 'Not Available'
    date = soup.find('h4').get_text(strip=True) if soup.find('h4') else 'Not Available'
    location = soup.find('h4').find_next('span').get_text(strip=True) if soup.find('h4') else 'Not Available'
    event_type = soup.find('a', class_='category').get_text(strip=True) if soup.find('a', class_='category') else 'Not Available'
    region = soup.find_all('a', class_='category')[1].get_text(strip=True) if len(soup.find_all('a', class_='category')) > 1 else 'Not Available'
    lat, lon = get_lat_lon(location)
    return {
        'Name': name,
        'Date': date,
        'Location': location,
        'Type': event_type,
        'Region': region,
        'Latitude': lat,
        'Longitude': lon
    }


In [19]:
import requests

# Function to get latitude and longitude for a given location
def get_lat_lon(location):
    base_url = "https://nominatim.openstreetmap.org/search.php"
    query_params = {
        "q": location,
        "format": "jsonv2"
    }
    response = requests.get(base_url, params=query_params)
    data = response.json()

    if data and isinstance(data, list) and len(data) > 0:
        return data[0].get('lat'), data[0].get('lon')
    else:
        return None, None


In [21]:
import requests

# Function to get weather data based on latitude and longitude
def get_weather_data(lat, lon):
    point_url = f"https://api.weather.gov/points/{lat},{lon}"
    point_res = requests.get(point_url)

    if point_res.status_code == 200:
        point_data = point_res.json()
        if 'properties' in point_data and 'forecast' in point_data['properties']:
            forecast_url = point_data['properties']['forecast']
            forecast_res = requests.get(forecast_url)
            forecast_data = forecast_res.json()

            if 'properties' in forecast_data and 'periods' in forecast_data['properties']:
                periods = forecast_data['properties']['periods']
                return periods
            else:
                return 'Forecast data not available'
        else:
            return 'Forecast URL not found'
    else:
        return 'Point data not available'


In [ ]:

event_details = []

for url in event_links:
    time.sleep(1)
    details = scrape_event_details(url)
    if details['Latitude'] and details['Longitude']:
        weather_forecast = get_weather_data(details['Latitude'], details['Longitude'])
        details['Weather'] = weather_forecast
    else:
        details['Weather'] = 'Weather data not available'
    event_details.append(details)

# Write the data to a CSV file
csv_file = 'events.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Name', 'Date', 'Location', 'Type', 'Region', 'Latitude', 'Longitude', 'Weather'])
    writer.writeheader()
    for event in event_details:
        writer.writerow(event)

print(f"Data written to {csv_file}")
